In [3]:
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from ipywidgets import interact, FloatSlider

# Define the function that simulates the system
def simulate_pid_with_delay(Kp, Ki, Kd, delay_seconds):
    """
    Simulates a closed-loop control system with a full PID controller and a
    transport delay.
    
    Args:
        Kp (float): Proportional gain.
        Ki (float): Integral gain.
        Kd (float): Derivative gain.
        tau (float): Time constant of the first-order system.
    """
    # System parameters
    tau=5 # Time constant of the first-order system
    sat_limit=10 # The saturation limit for the controller output

    # Simulation parameters
    t_final = 25
    dt = 0.05  # Time step
    t_vals = np.arange(0, t_final, dt)
    
    # Store past values for the delay
    delay_steps = int(delay_seconds / dt)
    if delay_steps == 0:
        delay_steps = 1
    
    # A deque is an efficient way to store a fixed-size history
    delayed_input = deque([0.0] * delay_steps, maxlen=delay_steps)
    
    # State variables
    y_system = np.zeros_like(t_vals)
    y_current = 0.0
    
    # PID controller state variables
    e_integral = 0.0
    e_prev = 0.0
    
    # Main simulation loop
    for i in range(len(t_vals)):
        # Desired setpoint (step input at t=0)
        r = 1.0
        
        # Calculate the error
        e = r - y_current
        
        # Calculate the integral term (using a rectangular approximation)
        e_integral += e * dt
        
        # Calculate the derivative term (using finite difference)
        e_derivative = (e - e_prev) / dt
        
        # PID control signal
        u_unlimited = Kp * e + Ki * e_integral + Kd * e_derivative
        
        # Apply saturation
        u_saturated = max(-sat_limit, min(sat_limit, u_unlimited))
        
        # Update the delayed input queue with the current saturated control signal
        delayed_input.append(u_saturated)
        
        # The input to the process is the delayed signal
        u_process = delayed_input[0]
        
        # Non-linear first-order system dynamics: dy/dt = (u_process - y) / tau
        dydt = (u_process - y_current) / tau
        
        # Euler integration to update the system state
        y_current += dydt * dt
        y_system[i] = y_current
        
        # Update previous error for the next derivative calculation
        e_prev = e
        
    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(t_vals, y_system, label=f'Kp={Kp}, Ki={Ki}, Kd={Kd}, Delay={delay_seconds}s')
    ax.set_title('PID Control with Transport Delay (Setpoint = 1)')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Output')
    ax.set_ylim(-0.5, 2.0)
    ax.grid(True)
    ax.legend()
    plt.show()

# Create the interactive sliders
interact(simulate_pid_with_delay,
         Kp=FloatSlider(min=0.1, max=20.0, step=0.1, value=5.0, description='Kp'),
         Ki=FloatSlider(min=0.0, max=3.0, step=0.1, value=0.0, description='Ki'),
         Kd=FloatSlider(min=0.0, max=5.0, step=0.1, value=0.0, description='Kd'),
         delay_seconds=FloatSlider(min=0.0, max=2, step=0.1, value=1.0, description='Delay (s)'),
        )

interactive(children=(FloatSlider(value=5.0, description='Kp', max=20.0, min=0.1), FloatSlider(value=0.0, desc…

<function __main__.simulate_pid_with_delay(Kp, Ki, Kd, delay_seconds)>